In [6]:
import pickle
import pandas as pd
import nltk
from bs4 import BeautifulSoup
import string
import re

In [7]:
input_file = open('./models.pkl', 'rb')
models = pickle.load(input_file)
input_file.close()

In [8]:
#function to perform text preprocessing on an input example

def preprocess(title, body):
    body = BeautifulSoup(body).get_text()
    title = title.lower()
    body = body.lower()
    title = re.sub(r'\d+', '', title)
    body = re.sub(r'\d+', '', body)
    title = title.translate(title.maketrans('', '', string.punctuation))
    body = body.translate(body.maketrans('', '', string.punctuation))
    title = title.strip()
    body = body.strip()
    stopwords = set(nltk.corpus.stopwords.words('english'))
    title = " ".join([word for word in nltk.tokenize.word_tokenize(title) if not word in stopwords])
    body = " ".join([word for word in nltk.tokenize.word_tokenize(body) if not word in stopwords])
    stemmer = nltk.stem.SnowballStemmer('english')
    title = " ".join([stemmer.stem(word) for word in nltk.tokenize.word_tokenize(title)])
    body = " ".join([stemmer.stem(word) for word in nltk.tokenize.word_tokenize(body)])
    return title, body

In [10]:
#predict the output labels for a new question

title = input("Enter Question Title: ")
body = input("Enter Question Body: ")
for tag, model in models.items():
    x = pd.DataFrame(columns=model['features'], index=range(1))
    for word in model['features']:
        x[word] = 0
    for word in nltk.tokenize.word_tokenize(title):
        if word in model['features']:
            x.loc[x.index[0], word] = 1
    for word in nltk.tokenize.word_tokenize(body):
        if word in model['features']:
            x.loc[x.index[0], word] = 1
    print(tag, model['model'].predict(x))